In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import os
from collections import Counter
from imblearn.over_sampling import SMOTE

In [4]:
from feature_selection import forwardFeatureSelection

from GenLinCFA import GenLinCFA

from aux_GenLinCFA import *
import random

In [5]:
def compute_stat_mean(values):
  mean = np.mean(values)
  std_dev = 1.96*np.std(values)/np.sqrt(len(values))
  return round(mean,2), round(std_dev,2)

In [ ]:
compute_stat_mean([0.64, 0.64, 0.65, 0.65, 0.65])

(0.65, 0.0)

In [ ]:
def compute_random_seeds(num_seeds, limit):
  random.seed(12)
  randomlist = []
  for i in range(0,num_seeds):
    n = random.randint(1,limit)
    randomlist.append(n)
  return randomlist

# GenLinCFA aggregations original target and standardized



## Temp + Prec

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
path_target = './csv/'
path_features = './features_allvalues/'
randomlist = compute_random_seeds(10, 100)
randomlist

[61, 35, 85, 68, 86, 45, 19, 49, 2, 48]

In [ ]:
destination_folder = './GenLinCFA/temp_prec_original_target_shuffle/'
plots_folder = './GenLinCFA/for_plots_original_target/'

# basins = ['Emiliani1','Emiliani2', 'Garda_Mincio']
basins = ['Adda','Lambro_Olona', 'Oglio_Iseo', 'Ticino']

threshold = 0.5

for basin in basins:
  selected_colnames_CMI5 = []
  outputs = []
  for i, random_seed in enumerate(randomlist):
    if i < 5:
      print('####################' + basin + '####################')
      target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target_binary(
          '',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', 
          path=path_target+basin+'.csv', threshold = threshold, nopeaks = False)
      eps = 0.37
      actual_path = path_features+basin+'_aggreg.csv'
      output, aggregate_trainVal, aggregate_test, target_df_trainVal_res = aggregate_unfolded_data_SMOTE(actual_path,['cyclostationary_mean_tg', 
                                                                                'cyclostationary_mean_tg_1w',
                                                                                'cyclostationary_mean_tg_4w', 
                                                                                'cyclostationary_mean_tg_8w',
                                                                                'cyclostationary_mean_tg_12w', 
                                                                                'cyclostationary_mean_tg_16w',
                                                                                'cyclostationary_mean_tg_24w',
                                                                                'cyclostationary_mean_rr', 
                                                                                'cyclostationary_mean_rr_1w',
                                                                                'cyclostationary_mean_rr_4w', 
                                                                                'cyclostationary_mean_rr_8w',
                                                                                'cyclostationary_mean_rr_12w', 
                                                                                'cyclostationary_mean_rr_16w',
                                                                                'cyclostationary_mean_rr_24w'
                                                                                ],
                                                                          target_df_trainVal, eps=eps,
                                                                          max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                          curr_seed=random_seed, shuffle=True)
      target_df_trainVal_res = target_df_trainVal_res.to_frame(name="mean_std")
      res = {
                "delta" : [], 
                "numSelected" : [], 
                "selectedFeatures" : [] 
            }
      
      res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal_res),res,10,1)
      
      selectedFeatures='selectedFeatures'
      print(f'\n{res[selectedFeatures]}\n')
      selected_colnames_CMI = aggregate_trainVal.columns[res['selectedFeatures']]

      print('\nFull model and selected features with CMI\n')
      compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal_res, target_df_test, selected_colnames_CMI)

      print('\nFull model and best 5 selected features with CMI\n')
      compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal_res, target_df_test, selected_colnames_CMI[0:5])

      outputs.append(output)
      selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames_CMI[0:5]].columns.values)

      train_string = destination_folder + basin + '_' + str(i) + '_genLinCFA_best5_CMI_train.csv'
      val_string = destination_folder + basin + '_' + str(i) + '_genLinCFA_best5_CMI_val.csv'
      test_string = destination_folder + basin + '_' + str(i) + '_genLinCFA_best5_CMI_test.csv'

      X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames_CMI[0:5]]
      X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames_CMI[0:5]]
      X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
      X_test_CMI5 = aggregate_test.loc[:,selected_colnames_CMI[0:5]]
              
      X_train_CMI5.to_csv(train_string, index=False)
      X_validation_CMI5.to_csv(val_string, index=False)
      X_test_CMI5.to_csv(test_string, index=False)

      log_regr = LogisticRegression(solver='lbfgs', random_state = 42)
      log_regr.fit(X_train_validation_CMI5.values, target_df_trainVal_res['mean_std'])
      print("Train accuracy logregr CMI best 5 for shuffle n." + str(i) + ": ", round(log_regr.score(X_train_validation_CMI5.values, target_df_trainVal_res['mean_std']),3))
      print("Test accuracy logregr CMI best 5 for shuffle n." + str(i) + ": ", round(log_regr.score(X_test_CMI5.values, target_df_test['mean_std']),3), "\n")

  output_string = plots_folder + basin + '_aggregations.npy'
  sel_col_string = plots_folder + basin + '_chosen_features.npy'
  np.save(output_string, outputs)
  np.save(sel_col_string, selected_colnames_CMI5)

####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1         0
1    2001-01-13  0.380618    0.43  2001     2         0
2    2001-01-21  0.341985    0.38  2001     3         0
3    2001-01-29  0.322044    0.35  2001     5         0
4    2001-02-06  0.354954    0.40  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48         0
407  2009-12-05  0.409921    0.46  2009    49         0
408  2009-12-13  0.472087    0.53  2009    50         0
409  2009-12-21  0.324728    0.00  2009    52         0
410  2009-12-29  0.086512    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Original dataset shape Counter({0: 425, 1: 214})
Resampled dataset shape Counter({0: 425, 1: 425})
Feature: cyclostationary-mean-tg

Number of features: 92

58
54
50
45
38
28
19
16
13
8


/usr/local/lib/python3.9/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Lambro_Olona####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.369625    0.45  2001     1         0
1    2001-01-13  0.429563    0.43  2001     2         0
2    2001-01-21  0.470784    0.48  2001     3         0
3    2001-01-29  0.370358    0.37  2001     5         0
4    2001-02-06  0.372263    0.37  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.402059    0.40  2009    48         0
407  2009-12-05  0.389658    0.39  2009    49         0
408  2009-12-13  0.545184    0.56  2009    50         1
409  2009-12-21  0.447916    0.55  2009    52         0
410  2009-12-29  0.277300    0.32  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Original dataset shape Counter({0: 414, 1: 225})
Resampled dataset shape Counter({0: 414, 1: 414})
Feature: cyclostationary-mean-tg

Number of features: 55

55
55
55
55
55
55
52


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7741545893719807, test score: 0.6228070175438597
Aggregate classification train score with FS: 0.711352657004831, test score: 0.6885964912280702

Full model and best 5 selected features with CMI



...

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Full aggregate classification train score: 0.7717391304347826, test score: 0.6271929824561403
Aggregate classification train score with FS: 0.711352657004831, test score: 0.6885964912280702
Train accuracy logregr CMI best 5 for shuffle n.4:  0.714
Test accuracy logregr CMI best 5 for shuffle n.4:  0.689 

####################Oglio_Iseo####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.243674    0.26  2001     1         0
1    2001-01-13  0.424116    0.44  2001     2         0
2    2001-01-21  0.393786    0.39  2001     3         0
3    2001-01-29  0.314939    0.31  2001     5         0
4    2001-02-06  0.464902    0.48  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.465734    0.48  2009    48         0
407  2009-12-05  0.447390    0.47  2009    49         0
408  2009-12-13  0.556760    0.59  2009    50         1
409  2009-12-21  0.307880    0.00  2009    52         0
410  2009-

/usr/local/lib/python3.9/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Original dataset shape Counter({0: 456, 1: 183})
Resampled dataset shape Counter({0: 456, 1: 456})
Feature: cyclostationary-mean-tg

Number of features: 92

19
17
10
Number of aggregated features: 10

Feature: cyclostationary-mean-tg-1w

Number of features: 92

41
37
32
30
25
23
17
12
7
Number of aggregated features: 7

Feature: cyclostationary-mean-tg-4w

Number of features: 92

1
1
1
1
1
1
1
1
1
1
1
1
5
Number of aggregated features: 5

Feature: cyclostationary-mean-tg-8w

Number of features: 92

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
Number of aggregated features: 4

Feature: cyclostationary-mean-tg-12w

Number of features: 92

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
4
Number of aggregated features: 4

Feature: cyclostationary-mean-tg-16w

Number of features: 92

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
5
Number of aggregated features: 5

Feature: cyclostatio

/usr/local/lib/python3.9/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
# basins = ['Emiliani1','Emiliani2', 'Garda_Mincio']
basins = ['Adda','Lambro_Olona', 'Oglio_Iseo', 'Ticino']
# basins = ['Dora','Piemonte_Sud', 'Piemonte_Nord']

destination_folder = './GenLinCFA/temp_prec_shuffle/'
plots_folder = './GenLinCFA/for_plots/'

for basin in basins:
  selected_colnames_CMI5 = []
  outputs = []
  for i, random_seed in enumerate(randomlist):
    if i < 5:
      print('####################' + basin + '####################')
      target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', 
          path=path_target+basin+'.csv', threshold = None, nopeaks = False)
      eps = 0.37
      actual_path = path_features+basin+'_aggreg.csv'
      output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                                'cyclostationary_mean_tg_1w',
                                                                                'cyclostationary_mean_tg_4w', 
                                                                                'cyclostationary_mean_tg_8w',
                                                                                'cyclostationary_mean_tg_12w', 
                                                                                'cyclostationary_mean_tg_16w',
                                                                                'cyclostationary_mean_tg_24w',
                                                                                'cyclostationary_mean_rr', 
                                                                                'cyclostationary_mean_rr_1w',
                                                                                'cyclostationary_mean_rr_4w', 
                                                                                'cyclostationary_mean_rr_8w',
                                                                                'cyclostationary_mean_rr_12w', 
                                                                                'cyclostationary_mean_rr_16w',
                                                                                'cyclostationary_mean_rr_24w'
                                                                                ],
                                                                          target_df_trainVal, eps=eps,
                                                                          max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                          curr_seed=random_seed, shuffle=True)
      res = {
                "delta" : [], 
                "numSelected" : [], 
                "selectedFeatures" : [] 
            }
      
      res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
      
      selectedFeatures='selectedFeatures'
      print(f'\n{res[selectedFeatures]}\n')
      selected_colnames_CMI = aggregate_trainVal.columns[res['selectedFeatures']]

      print('\nFull model and selected features with CMI\n')
      compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames_CMI)

      print('\nFull model and best 5 selected features with CMI\n')
      compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames_CMI[0:5])

      outputs.append(output)
      
      selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames_CMI[0:5]].columns.values)

      train_string = destination_folder + basin + '_' + str(i) + '_genLinCFA_best5_CMI_train.csv'
      val_string = destination_folder + basin + '_' + str(i) + '_genLinCFA_best5_CMI_val.csv'
      test_string = destination_folder + basin + '_' + str(i) + '_genLinCFA_best5_CMI_test.csv'

      X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames_CMI[0:5]]
      X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames_CMI[0:5]]
      X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
      X_test_CMI5 = aggregate_test.loc[:,selected_colnames_CMI[0:5]]
              
      X_train_CMI5.to_csv(train_string, index=False)
      X_validation_CMI5.to_csv(val_string, index=False)
      X_test_CMI5.to_csv(test_string, index=False)

      log_regr = LogisticRegression(solver='lbfgs', random_state = 42)
      log_regr.fit(X_train_validation_CMI5.values, target_df_trainVal['mean_std'])
      print("Train accuracy logregr CMI best 5 for shuffle n." + str(i) + ": ", round(log_regr.score(X_train_validation_CMI5.values, target_df_trainVal['mean_std']),3))
      print("Test accuracy logregr CMI best 5 for shuffle n." + str(i) + ": ", round(log_regr.score(X_test_CMI5.values, target_df_test['mean_std']),3), "\n")

  output_string = plots_folder + basin + '_aggregations.npy'
  sel_col_string = plots_folder + basin + '_chosen_features.npy'
  np.save(output_string, outputs)
  np.save(sel_col_string, selected_colnames_CMI5)

####################Dora####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.010645    0.00  2001     1         0
1    2001-01-13  0.206769    0.00  2001     2         0
2    2001-01-21  0.267313    0.00  2001     3         0
3    2001-01-29  0.240836    0.20  2001     5         0
4    2001-02-06  0.193417    0.15  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.230073    0.25  2009    48         0
407  2009-12-05  0.243632    0.24  2009    49         0
408  2009-12-13  0.251111    0.00  2009    50         0
409  2009-12-21  0.099246    0.00  2009    52         0
410  2009-12-29  0.064990    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Feature: cyclostationary_mean_tg

Number of features: 44

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
Number of aggregated features: 5

Feature: cyclostationary_mea

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.6682316118935837, test score: 0.5043859649122807
Aggregate classification train score with FS: 0.6165884194053208, test score: 0.5263157894736842

Full model and best 5 selected features with CMI



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.6682316118935837, test score: 0.5043859649122807
Aggregate classification train score with FS: 0.6165884194053208, test score: 0.5263157894736842
Train accuracy logregr CMI best 5 for shuffle n.0:  0.618
Test accuracy logregr CMI best 5 for shuffle n.0:  0.531 

####################Dora####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.010645    0.00  2001     1         0
1    2001-01-13  0.206769    0.00  2001     2         0
2    2001-01-21  0.267313    0.00  2001     3         0
3    2001-01-29  0.240836    0.20  2001     5         0
4    2001-02-06  0.193417    0.15  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.230073    0.25  2009    48         0
407  2009-12-05  0.243632    0.24  2009    49         0
408  2009-12-13  0.251111    0.00  2009    50         0
409  2009-12-21  0.099246    0.00  2009    52         0
410  2009-12-29

...

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7355242566510172, test score: 0.5877192982456141
Aggregate classification train score with FS: 0.6901408450704225, test score: 0.6359649122807017

Full model and best 5 selected features with CMI



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7355242566510172, test score: 0.5877192982456141
Aggregate classification train score with FS: 0.6885758998435054, test score: 0.6271929824561403
Train accuracy logregr CMI best 5 for shuffle n.2:  0.69
Test accuracy logregr CMI best 5 for shuffle n.2:  0.632 

####################Piemonte_Nord####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278983    0.00  2001     1         0
1    2001-01-13  0.494910    0.51  2001     2         1
2    2001-01-21  0.496092    0.51  2001     3         1
3    2001-01-29  0.427992    0.43  2001     5         0
4    2001-02-06  0.400512    0.41  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.363952    0.37  2009    48         0
407  2009-12-05  0.400487    0.40  2009    49         0
408  2009-12-13  0.506771    0.52  2009    50         1
409  2009-12-21  0.387530    0.53  2009    52         0
410  20

/usr/local/lib/python3.9/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# GenLinCFA aggregations original target and std, moving avg



## Temp + Prec

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import random

path_target = './csv/'
path_features = './features_allvalues/'
random.seed(12)

randomlist = []
for i in range(0,10):
  n = random.randint(1,100)
  randomlist.append(n)
randomlist

[61, 35, 85, 68, 86, 45, 19, 49, 2, 48]

In [ ]:
basins = ['Emiliani1','Emiliani2', 'Garda_Mincio']

destination_folder = './GenLinCFA/temp_prec_moving_avg_2_shuffle/'
plots_folder = './GenLinCFA/for_plots_moving_avg_2/'

for basin in basins:
  selected_colnames_CMI5 = []
  outputs = []
  for i, random_seed in enumerate(randomlist):
    if i < 5:
      print('####################' + basin + '####################')
      target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', 
          path=path_target+basin+'.csv', threshold = None, nopeaks = False, window_size = 2)
      eps = 0.37
      actual_path = path_features+basin+'_aggreg.csv'
      output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                                'cyclostationary_mean_tg_1w',
                                                                                'cyclostationary_mean_tg_4w', 
                                                                                'cyclostationary_mean_tg_8w',
                                                                                'cyclostationary_mean_tg_12w', 
                                                                                'cyclostationary_mean_tg_16w',
                                                                                'cyclostationary_mean_tg_24w',
                                                                                'cyclostationary_mean_rr', 
                                                                                'cyclostationary_mean_rr_1w',
                                                                                'cyclostationary_mean_rr_4w', 
                                                                                'cyclostationary_mean_rr_8w',
                                                                                'cyclostationary_mean_rr_12w', 
                                                                                'cyclostationary_mean_rr_16w',
                                                                                'cyclostationary_mean_rr_24w'
                                                                                ],
                                                                          target_df_trainVal, eps=eps,
                                                                          max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                          curr_seed=random_seed, shuffle=True)
      res = {
                "delta" : [], 
                "numSelected" : [], 
                "selectedFeatures" : [] 
            }
      
      res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
      
      selectedFeatures='selectedFeatures'
      print(f'\n{res[selectedFeatures]}\n')
      selected_colnames_CMI = aggregate_trainVal.columns[res['selectedFeatures']]

      print('\nFull model and selected features with CMI\n')
      compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames_CMI)

      print('\nFull model and best 5 selected features with CMI\n')
      compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames_CMI[0:5])

      outputs.append(output)
      
      selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames_CMI[0:5]].columns.values)

      train_string = destination_folder + basin + '_' + str(i) + '_genLinCFA_best5_CMI_train.csv'
      val_string = destination_folder + basin + '_' + str(i) + '_genLinCFA_best5_CMI_val.csv'
      test_string = destination_folder + basin + '_' + str(i) + '_genLinCFA_best5_CMI_test.csv'

      X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames_CMI[0:5]]
      X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames_CMI[0:5]]
      X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
      X_test_CMI5 = aggregate_test.loc[:,selected_colnames_CMI[0:5]]
              
      X_train_CMI5.to_csv(train_string, index=False)
      X_validation_CMI5.to_csv(val_string, index=False)
      X_test_CMI5.to_csv(test_string, index=False)

      log_regr = LogisticRegression(solver='lbfgs', random_state = 42)
      log_regr.fit(X_train_validation_CMI5.values, target_df_trainVal['mean_std'])
      print("Train accuracy logregr CMI best 5 for shuffle n." + str(i) + ": ", round(log_regr.score(X_train_validation_CMI5.values, target_df_trainVal['mean_std']),3))
      print("Test accuracy logregr CMI best 5 for shuffle n." + str(i) + ": ", round(log_regr.score(X_test_CMI5.values, target_df_test['mean_std']),3), "\n")

  output_string = plots_folder + basin + '_aggregations.npy'
  sel_col_string = plots_folder + basin + '_chosen_features.npy'
  np.save(output_string, outputs)
  np.save(sel_col_string, selected_colnames_CMI5)

####################Emiliani1####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.379890    0.50  2001     1         0
1    2001-01-13  0.431284    0.58  2001     2         0
2    2001-01-21  0.499469    0.59  2001     3         1
3    2001-01-29  0.475340    0.50  2001     5         1
4    2001-02-06  0.464613    0.54  2001     6         1
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.400871    0.43  2009    48         0
407  2009-12-05  0.487232    0.57  2009    49         1
408  2009-12-13  0.539225    0.58  2009    50         1
409  2009-12-21  0.413387    0.00  2009    52         0
410  2009-12-29  0.161782    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Feature: cyclostationary_mean_tg

Number of features: 172

1
1
1
1
3
Number of aggregated features: 3

Feature: cyclostationary_mean_tg_1w

Number of features: 172

Number of aggreg

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.8231611893583725, test score: 0.6622807017543859
Aggregate classification train score with FS: 0.7433489827856025, test score: 0.75

Full model and best 5 selected features with CMI



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.8231611893583725, test score: 0.6622807017543859
Aggregate classification train score with FS: 0.7496087636932708, test score: 0.75
Train accuracy logregr CMI best 5 for shuffle n.3:  0.75
Test accuracy logregr CMI best 5 for shuffle n.3:  0.75 

####################Emiliani1####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.379890    0.50  2001     1         0
1    2001-01-13  0.431284    0.58  2001     2         0
2    2001-01-21  0.499469    0.59  2001     3         1
3    2001-01-29  0.475340    0.50  2001     5         1
4    2001-02-06  0.464613    0.54  2001     6         1
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.400871    0.43  2009    48         0
407  2009-12-05  0.487232    0.57  2009    49         1
408  2009-12-13  0.539225    0.58  2009    50         1
409  2009-12-21  0.413387    0.00  2009    52         0
410  2009-12-29  0.161782 

...

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7902973395931142, test score: 0.706140350877193
Aggregate classification train score with FS: 0.729264475743349, test score: 0.6710526315789473
Train accuracy logregr CMI best 5 for shuffle n.3:  0.732
Test accuracy logregr CMI best 5 for shuffle n.3:  0.671 

####################Garda_Mincio####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.102270    0.00  2001     1         0
1    2001-01-13  0.278351    0.53  2001     2         0
2    2001-01-21  0.388973    0.32  2001     3         1
3    2001-01-29  0.312587    0.31  2001     5         0
4    2001-02-06  0.348197    0.44  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.354587    0.44  2009    48         0
407  2009-12-05  0.395667    0.47  2009    49         1
408  2009-12-13  0.378271    0.44  2009    50         0
409  2009-12-21  0.198864    0.00  2009    52         0
410  2009

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.8137715179968701, test score: 0.7105263157894737
Aggregate classification train score with FS: 0.5852895148669797, test score: 0.6096491228070176

Full model and best 5 selected features with CMI

Full aggregate classification train score: 0.8137715179968701, test score: 0.7105263157894737
Aggregate classification train score with FS: 0.5852895148669797, test score: 0.6096491228070176
Train accuracy logregr CMI best 5 for shuffle n.4:  0.582
Test accuracy logregr CMI best 5 for shuffle n.4:  0.623 



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# GenLinCFA results in binary classification

## Features and target - Single Task

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
destination_folder = './GenLinCFA/temp_prec_shuffle/'
plots_folder = './GenLinCFA/for_plots_original_target/'

In [ ]:
### original targets
basins = ['Emiliani1','Emiliani2','Garda_Mincio']
path_orig_targets = './csv/'

targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', 
          path=path_orig_targets+basin+'.csv', threshold = None, nopeaks = False, window_size = 2)
    targets_df_test[basin] = target_df_test['mean_std']
    targets_df_train[basin] = target_df_train['mean_std']
    targets_df_val[basin] = target_df_val['mean_std']
    targets_df_trainVal[basin] = pd.concat([targets_df_train[basin], targets_df_val[basin]])

target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.379890    0.50  2001     1         0
1    2001-01-13  0.431284    0.58  2001     2         0
2    2001-01-21  0.499469    0.59  2001     3         1
3    2001-01-29  0.475340    0.50  2001     5         1
4    2001-02-06  0.464613    0.54  2001     6         1
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.400871    0.43  2009    48         0
407  2009-12-05  0.487232    0.57  2009    49         1
408  2009-12-13  0.539225    0.58  2009    50         1
409  2009-12-21  0.413387    0.00  2009    52         0
410  2009-12-29  0.161782    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.214281    0.00  2001     1         0
1    2001-01-13  0.349509    0.52  2001     2         0
2    2001-01-21  0.475404    0.47  2001     3  

In [ ]:
def get_features (path_features, basin, train_name, val_name, test_name):
  features_train = pd.read_csv(path_features + basin + train_name)
  features_valid = pd.read_csv(path_features + basin + val_name)
  features_test = pd.read_csv(path_features + basin + test_name)
  
  features_train_validation = pd.concat([features_train, features_valid])
  return features_train.values, features_valid.values, features_train_validation.values, features_test.values

In [ ]:
basin = 'Emiliani1'

In [ ]:
# Temp Prec
path_features = './GenLinCFA/temp_prec_shuffle/'

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_4_genLinCFA_best5_CMI_train.csv', 
                                                                                     '_4_genLinCFA_best5_CMI_val.csv', '_4_genLinCFA_best5_CMI_test.csv')

## Single Task

In [ ]:
log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

# CMI best 5
log_regr.fit(X_train_validation_CMI5, targets_df_trainVal[basin])
print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, targets_df_trainVal[basin]),3))
print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, targets_df_test[basin]),3), "\n")

Train accuracy logistic regression CMI best 5:  0.718
Test accuracy logistic regression CMI best 5:  0.697 



## Features and target - Multi Task

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
### original targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_orig_targets = './csv/'

threshold = 0.5

targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                            max_test='2020-01-01', path=path_orig_targets+basin+'.csv', 
                                                                                            threshold = None, nopeaks = False)
    targets_df_test[basin] = target_df_test['mean_std']
    targets_df_train[basin] = target_df_train['mean_std']
    targets_df_val[basin] = target_df_val['mean_std']
    targets_df_trainVal[basin] = pd.concat([targets_df_train[basin], targets_df_val[basin]])

target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1         0
1    2001-01-13  0.380618    0.43  2001     2         0
2    2001-01-21  0.341985    0.38  2001     3         0
3    2001-01-29  0.322044    0.35  2001     5         0
4    2001-02-06  0.354954    0.40  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48         0
407  2009-12-05  0.409921    0.46  2009    49         0
408  2009-12-13  0.472087    0.53  2009    50         1
409  2009-12-21  0.324728    0.00  2009    52         0
410  2009-12-29  0.086512    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.010645    0.00  2001     1         0
1    2001-01-13  0.206769    0.00  2001     2         0
2    2001-01-21  0.267313    0.00  2001     3  

In [ ]:
### CMI best5 features for standardized target
basins = ['Emiliani1', 'Emiliani2', 'Garda_Mincio']
path_features = './GenLinCFA/temp_prec_moving_avg_2_shuffle/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_4_genLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_4_genLinCFA_best5_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_4_genLinCFA_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features for original target
# basins = ['Emiliani1', 'Emiliani2', 'Garda_Mincio']
basins = ['Dora', 'Piemonte_Nord', 'Piemonte_Sud']

# path_features = './GenLinCFA/temp_prec_original_target_shuffle/'

path_features = './GenLinCFA/temp_prec_shuffle/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_3_genLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_3_genLinCFA_best5_CMI_val.csv')
    train_val_temp = pd.concat((train_temp, val_temp))
    val_temp.reset_index(inplace = True, drop = True)
    test_temp = pd.read_csv(path_features+basin+'_3_genLinCFA_best5_CMI_test.csv')
    train_val_temp.reset_index(inplace = True, drop = True)
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_val_temp.iloc[:411,:]
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = train_val_temp.iloc[411:639,:]
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

best5_CMI_fulldf_val.reset_index(inplace = True, drop = True)

## Multi Task

In [ ]:
from sklearn.metrics import accuracy_score
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)
    
    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()
    
    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())
    
    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))

In [ ]:
MTL_scores(clust_basins=['Dora','Piemonte_Nord','Piemonte_Sud'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.5789473684210527
Piemonte_Nord
0.6535087719298246
Piemonte_Sud
0.631578947368421


In [ ]:
def MTL_scores_smote(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)
    
    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()
    
    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    clusterdf_train_val = pd.concat((clusterdf_train_withClass,clusterdf_val_withClass))
    target_df_train_val = pd.concat((targets_df_train_unfolded,targets_df_val_unfolded))

    sm = SMOTE(random_state=42)
    clusterdf_train_val_res, target_df_train_val_res = sm.fit_resample(clusterdf_train_val, target_df_train_val)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(clusterdf_train_val_res.values, target_df_train_val_res.values.ravel())
    
    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))
    #return clusterdf_train_val, clusterdf_train_val_res, target_df_train_val_res

In [ ]:
MTL_scores_smote(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.7368421052631579
Emiliani2
0.7192982456140351
Garda_Mincio
0.6842105263157895
